In [ ]:
import numpy as np
import pandas as pd
import os
import torch.nn as nn
from glob import glob
from PIL import Image
import matplotlib.pyplot as plt
from torchvision.utils import make_grid
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import ToTensor, Resize
from sklearn.model_selection import train_test_split

**Reading the Dataset**

In [ ]:
import os
import random 
from glob import glob
from pathlib import Path

DATASET_DIR="../input/dermnet-and-skin-disease/Merged Dataset"
TEST_DIR="../input/dermnet-and-skin-disease/Merged Dataset/test"
TRAIN_DIR="../input/dermnet-and-skin-disease/Merged Dataset/Train"


all_data = [y for x in os.walk(DATASET_DIR) for y in glob(os.path.join(x[0], '*jpg'))]
all_labels = [os.path.basename(os.path.dirname(x)) for x in all_data]
print(len(all_data))
all_data[:10]

**Classes with labels**

In [ ]:
uniqe_labels=set(all_labels)
print("number of classes = ",len(uniqe_labels))
print("Here is the classes :")
for label in uniqe_labels :
  print (label)

**Showing Image Samples from the dataset**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Sample 25 images from dataset
indices = np.random.randint(0, len(all_data), size=25)
images = [all_data[i] for i in indices]
labels = [all_labels[i] for i in indices]

# Plot the 25 images
plt.figure(figsize=(10,10))
for i in range(len(indices)):
    plt.subplot(5, 5, i + 1)
    image = mpimg.imread(images[i]) # Read image from disk
    plt.imshow(image)
    plt.title(labels[i])
    plt.axis('off')
    
plt.show()

**Loading and preprocessing the data**
**ResNet50 Model with final parameters**

In [ ]:
## define train test and validation generators 
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

BATCH_SIZE = 64
HEIGHT = 224
WIDTH = 224


# 1. Construct an instance of the `ImageDataGenerator` class
train_datagen =  ImageDataGenerator(
      preprocessing_function = tf.keras.applications.resnet50.preprocess_input,
      width_shift_range=0.4,
      height_shift_range=0.3,
      rotation_range=90,
      validation_split=0.2,
      zoom_range=0.2,
    )

# 2. Retrieve the iterator
train_generator = train_datagen.flow_from_directory(TRAIN_DIR, 
                             shuffle = True,
                            seed = 7,
                            target_size=(HEIGHT, WIDTH), 
                            batch_size=BATCH_SIZE,
                            color_mode='rgb',
                            class_mode='categorical',
                            subset='training'
                                                   )

In [ ]:
val_generator = train_datagen.flow_from_directory(
                             TRAIN_DIR, 
                             shuffle = True,
                            seed = 7,
                            target_size=(HEIGHT, WIDTH), 
                            batch_size=BATCH_SIZE,
                            color_mode='rgb',
                            class_mode='categorical',
                            subset='validation'
    )


In [ ]:
test_datagen =  ImageDataGenerator(
    preprocessing_function = tf.keras.applications.resnet50.preprocess_input
    )

# 2. Retrieve the iterator
test_generator = test_datagen.flow_from_directory(TEST_DIR, 
                                                    target_size=(HEIGHT, WIDTH), 
                                                    batch_size=BATCH_SIZE,
                                                   color_mode='rgb',
                                                    shuffle=False,
                                                    class_mode='categorical',
                                                  )

In [ ]:
from tensorflow.keras.applications.resnet50 import ResNet50
from keras.models import Model
from keras.layers import Dense, Flatten, Dropout

# Load model without classification head
base_model = ResNet50(include_top = False,
                   weights = 'imagenet',
                   input_shape = (HEIGHT, WIDTH, 3))

# Print base model summary
base_model.summary()

# Mark loaded layers as trainable
for layer in base_model.layers[25:45]:
  layer.trainable = True

# Adding new classifier layers
x = Flatten()(base_model.layers[-1].output)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)
output = Dense(23, activation='softmax')(x)

# Define new model
model_Res50 = Model(inputs = base_model.inputs, outputs = output)

# Print summary
model_Res50.summary()

# Compile
base_learning_rate = 0.0001
model_Res50.compile(optimizer = tf.keras.optimizers.Adam(lr = base_learning_rate),
            loss = 'categorical_crossentropy',
            metrics = ['accuracy'])

In [ ]:
import tensorflow as tf
INITIAL_EPOCHS = 10

history = model_Res50.fit(train_generator,
                    validation_data = val_generator,
                    epochs = INITIAL_EPOCHS)

**Plotting Graphs for ResNet model**

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

In [ ]:
##EValuting the model
model_Res50.evaluate(test_generator)

In [ ]:
predicions=model_Res50.predict(test_generator)

**Confusion Matrix**

In [ ]:
import numpy as np
predicted_classes=np.argmax(predicions,axis=1)  
true_classes=test_generator.classes  
test_labels=list(test_generator.class_indices.keys())
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt


cm = confusion_matrix(true_classes,predicted_classes)

cm_disp = ConfusionMatrixDisplay(confusion_matrix=cm,display_labels=test_labels)

cm_disp = ConfusionMatrixDisplay(confusion_matrix=cm)
fig, ax = plt.subplots(figsize=(10,10))
cm_disp.plot(ax=ax)

In [ ]:
from sklearn.metrics import classification_report
report=classification_report(true_classes, predicted_classes)
print(report)

**VGG Model with final hyperparameters**

In [ ]:
## define train test and validation generators 
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

BATCH_SIZE = 64
HEIGHT = 224
WIDTH = 224


# 1. Construct an instance of the `ImageDataGenerator` class
train_datagen =  ImageDataGenerator(
      preprocessing_function = tf.keras.applications.vgg16.preprocess_input,
      width_shift_range=0.4,
      height_shift_range=0.3,
      rotation_range=90,
      validation_split=0.2,
      zoom_range=0.2,
    )

# 2. Retrieve the iterator
train_generator = train_datagen.flow_from_directory(TRAIN_DIR, 
                             shuffle = True,
                            seed = 7,
                            target_size=(HEIGHT, WIDTH), 
                            batch_size=BATCH_SIZE,
                            color_mode='rgb',
                            class_mode='categorical',
                            subset='training'
                                                   )

In [ ]:
val_generator = train_datagen.flow_from_directory(
                             TRAIN_DIR, 
                             shuffle = True,
                            seed = 7,
                            target_size=(HEIGHT, WIDTH), 
                            batch_size=BATCH_SIZE,
                            color_mode='rgb',
                            class_mode='categorical',
                            subset='validation'

In [ ]:
test_datagen =  ImageDataGenerator(
    preprocessing_function = tf.keras.applications.vgg16.preprocess_input
    )

# 2. Retrieve the iterator
test_generator = test_datagen.flow_from_directory(TEST_DIR, 
                                                    target_size=(HEIGHT, WIDTH), 
                                                    batch_size=BATCH_SIZE,
                                                   color_mode='rgb',
                                                    shuffle=False,
                                                    class_mode='categorical',
                                                  )

In [ ]:
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.layers import Dense, Flatten, Dropout

# Load model without classification head
base_model = VGG16(include_top = False,
                   weights = 'imagenet',
                   input_shape = (HEIGHT, WIDTH, 3))

# Print base model summary
base_model.summary()

# Mark loaded layers as not trainable

# You can choose to fine-tune some of the final layers:
for layer in base_model.layers[8:14]:
    layer.trainable = True

# You can pick which layers are trainable and which are not:
 #base_model.get_layer('block1_conv1').trainable = False


# Add new classifier layers
x = Flatten()(base_model.layers[-1].output)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)
output = Dense(23, activation='softmax')(x)

# Define new model
model = Model(inputs = base_model.inputs, outputs = output)

# Print summary
model.summary()

# Compile
base_learning_rate = 0.0001
model.compile(optimizer = tf.keras.optimizers.RMSprop(lr = base_learning_rate),
            loss = 'binary_crossentropy',
            metrics = ['accuracy'])

In [ ]:
model.evaluate(test_generator)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()


In [ ]:
predicions=model.predict(test_generator)

In [ ]:
predicted_classes=np.argmax(predicions,axis=1)  
true_classes=test_generator.classes  
test_labels=list(test_generator.class_indices.keys())
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

cm = confusion_matrix(true_classes,predicted_classes)

cm_disp = ConfusionMatrixDisplay(confusion_matrix=cm,display_labels=test_labels)

cm_disp = ConfusionMatrixDisplay(confusion_matrix=cm)
fig, ax = plt.subplots(figsize=(15,15))
cm_disp.plot(ax=ax)

In [ ]:
from sklearn.metrics import classification_report
report=classification_report(true_classes, predicted_classes)
print(report)

**Xception Model with final Parameters**

In [ ]:
## define train test and validation generators modelmodel_Res50
BATCH_SIZE = 64
HEIGHT = 224
WIDTH = 224


# 1. Construct an instance of the `ImageDataGenerator` class
train_datagen =  ImageDataGenerator(
      preprocessing_function = tf.keras.applications.xception.preprocess_input,
      width_shift_range=0.4,
      height_shift_range=0.3,
      rotation_range=90,
      validation_split=0.2,
      zoom_range=0.2,
     
    )

# 2. Retrieve the iterator
train_generator = train_datagen.flow_from_directory(TRAIN_DIR, 
                             shuffle = True,
                            seed = 7,
                            target_size=(HEIGHT, WIDTH), 
                            batch_size=BATCH_SIZE,
                            color_mode='rgb',
                            class_mode='categorical',
                            subset='training'
                                                   )

In [ ]:
val_generator = train_datagen.flow_from_directory(
                             TRAIN_DIR, 
                             shuffle = True,
                            seed = 7,
                            target_size=(HEIGHT, WIDTH), 
                            batch_size=BATCH_SIZE,
                            color_mode='rgb',
                            class_mode='categorical',
                            subset='validation'
    )


In [ ]:
test_datagen =  ImageDataGenerator(
    preprocessing_function = tf.keras.applications.xception.preprocess_input
    )

# 2. Retrieve the iterator
test_generator = test_datagen.flow_from_directory(TEST_DIR, 
                                                    target_size=(HEIGHT, WIDTH), 
                                                    batch_size=BATCH_SIZE,
                                                   color_mode='rgb',
                                                    shuffle=False,
                                                    class_mode='categorical',
                                                  )

In [ ]:
from tensorflow.keras.applications.xception import Xception
from keras.models import Model
from keras.layers import Dense, Flatten, Dropout




# Load model without classification head
base_model = Xception(include_top = False,
                   weights = 'imagenet',
                   input_shape = (HEIGHT, WIDTH, 3))

# Print base model summary
base_model.summary()

# Mark loaded layers as not trainable
for layer in base_model.layers[30:60]:
  layer.trainable = True

       
# Adding new classifier layers
x = Flatten()(base_model.layers[-1].output)
x = Dense(512, activation='relu')(x)
x = Dropout(0.7)(x)
x = Dense(256, activation='relu')(x)
output = Dense(23, activation='softmax')(x)

# Define new model
model_Xception = Model(inputs = base_model.inputs, outputs = output)

# Print summary
model_Xception.summary()

# Compile
base_learning_rate = 0.0001
model_Xception.compile(optimizer = tf.keras.optimizers.Adam(lr = base_learning_rate),
            loss = 'categorical_crossentropy',
            metrics = ['accuracy'])

In [ ]:
INITIAL_EPOCHS =10

history = model_Xception.fit(train_generator,
                    validation_data = val_generator,
                    epochs = INITIAL_EPOCHS)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()


In [ ]:
predicions=model_Xception.predict(test_generator)


In [ ]:
import numpy as np
predicted_classes=np.argmax(predicions,axis=1)  
true_classes=test_generator.classes  
test_labels=list(test_generator.class_indices.keys())
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
cm = confusion_matrix(true_classes,predicted_classes)
cm_disp = ConfusionMatrixDisplay(confusion_matrix=cm,display_labels=test_labels)
cm_disp = ConfusionMatrixDisplay(confusion_matrix=cm)
fig, ax = plt.subplots(figsize=(10,10))
cm_disp.plot(ax=ax)

In [ ]:
from sklearn.metrics import classification_report
report=classification_report(true_classes, predicted_classes)
print(report)